<a href="https://colab.research.google.com/github/QoutiOussama13/GiksGarden/blob/main/giksGarden2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The vision+text models

To-do
*   Combine the vision and text model in the agent
*   Make the vision as a tool in the agent extractor
*   add this to the UI



In [ ]:
!pip install -q langchain langchain-openai openai

In [ ]:
!pip install --quiet chromadb

In [87]:
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_community.vectorstores import Chroma
from typing import Annotated, Sequence, TypedDict,Dict,Tuple
from langchain.agents import initialize_agent, Tool,AgentExecutor
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
import chromadb

In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

## making the chain tool

In [ ]:
class Config():
    """
    Contains the configuration of the LLM.
    """
    model = 'gpt-3.5-turbo'
    llm = ChatOpenAI(temperature=0, model=model)

Test with random plant to se how the agent will behave

In [72]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Delphinium = """
In the corner of the garden, nestled against the weathered wooden fence,
stands a vibrant Blue Delphinium (Delphinium elatum).
Planted on May 12th, 2023, this majestic perennial boasts spires of deep indigo blossoms,
 reaching towards the sky like miniature towers. Its lush green foliage provides a striking contrast to the azure petals,
creating a captivating focal point in the garden. Preferring full sun and well-drained soil,
this resilient plant attracts pollinators like bees and butterflies,
adding both beauty and biodiversity to the outdoor space.  """

In [158]:
chroma_client = chromadb.Client()

In [159]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter

# load the document and split it into chunks
loader = TextLoader("/content/drive/MyDrive/LEAP oasis hackathon/delphinium.txt")
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=3000, chunk_overlap=300)
docs = text_splitter.split_documents(documents)


# load it into Chroma
db = Chroma.from_documents(docs, embeddings)

In [160]:
cfg = Config()
qa = RetrievalQA.from_chain_type(
    llm=cfg.llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

retrieval qa chain


In [161]:
qa.invoke('What are the characteristic of my plant?')

{'query': 'What are the characteristic of my plant?',
 'result': 'Your plant, the Blue Delphinium (Delphinium elatum), is a stunning perennial known for its towering spires of deep indigo blossoms. It belongs to the buttercup family (Ranunculaceae) and is native to the Northern Hemisphere. The plant typically grows quite tall, reaching heights of up to 6 feet or more, making it an excellent choice for adding vertical interest to garden landscapes. The flowers bloom in early to midsummer, densely packed along elongated stalks, creating a striking display of color and texture. Blue Delphiniums prefer full sun and well-drained soil, attracting pollinators like bees and butterflies. They are favored for their vibrant hues, elegant appearance, and cultural symbolism associated with qualities like dignity and grace. Additionally, they have been historically used for medicinal purposes, but caution is advised due to the presence of toxic alkaloids in some species.'}

Setting up the memroy for the agent

In [162]:
def setup_memory() -> Tuple[Dict, ConversationBufferMemory]:
    """
    Sets up memory for the  agent.
    :return a tuple with the agent keyword pairs and the conversation memory.
    """
    agent_kwargs = {
        "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
    }
    memory = ConversationBufferMemory(memory_key="memory", return_messages=True)
    return agent_kwargs, memory

In [169]:
def setup_agent() -> AgentExecutor:
    """
    Sets up the tools for a function based chain.
    We have here the following tools:

    """
    cfg = Config()
    tools = [
        Tool(
            name="knowledge base",
            func=qa.run,
            description="Used when you need information or have questions about the plant."
        ),
        Tool(
        name='web search',
        func=TavilySearchResults().run,
        description='''use this tool when you can't find the content in the knowledge base and you need more advenced search functionalities that are related to the user query '''
        ),
    ]
    agent_kwargs, memory = setup_memory()

    return initialize_agent(
        tools,
        cfg.llm,
        verbose=False,
        agent_kwargs=agent_kwargs,
        memory=memory
    )

In [170]:
agent = setup_agent()

In [91]:
import base64
import requests

def generate_caption(api_key, image_path):
    # Function to encode the image
    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # Getting the base64 string
    base64_image = encode_image(image_path)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4-vision-preview",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Please provide a detailed description of the image, including the possible organisms it may feature, the time of day, and any relevant details. Remember, if the user provides information unrelated to agriculture, plants, gardens, etc., simply respond with 'nothing'."
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 300
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    return response.json()['choices'][0]['message']['content']

In [92]:
api_key = os.environ['OPENAI_API_KEY']
image_path = "/content/delphinium_insect.png"
caption = generate_caption(api_key, image_path)
caption

"The image depicts a close-up view of a green leaf with numerous holes and some apparent pest damage. There's a visible cluster of small, spherical, pale eggs, possibly insect eggs, adhered to the surface of the leaf on the right side. The holes across the leaf suggest it has been fed on by an insect or a group of insects, such as caterpillars or beetles. The presence of eggs implies that an insect has chosen this plant to lay its eggs, which will likely lead to further infestation once they hatch.\n\nThe leaf itself looks to be from a herbaceous plant, with prominent veins and serrated edges. Additionally, there are tiny, hair-like structures along the leaf edges and its surface, which are characteristic of certain plant species. The fine webbing around the eggs and holes may indicate the presence of spider mites or other pests that produce webbing. The time of day or the specific type of plant can't be determined from this image, as there is no contextual environment depicted that wo

In [93]:
api_key = os.environ['OPENAI_API_KEY']
image_path = "/content/image_2024-03-05_163437730.png"
caption2 = generate_caption(api_key, image_path)
caption2

'nothing'

In [171]:
'I started noticing these insects on my plant what should I do?'+ caption

"I started noticing these insects on my plant what should I do?The image depicts a close-up view of a green leaf with numerous holes and some apparent pest damage. There's a visible cluster of small, spherical, pale eggs, possibly insect eggs, adhered to the surface of the leaf on the right side. The holes across the leaf suggest it has been fed on by an insect or a group of insects, such as caterpillars or beetles. The presence of eggs implies that an insect has chosen this plant to lay its eggs, which will likely lead to further infestation once they hatch.\n\nThe leaf itself looks to be from a herbaceous plant, with prominent veins and serrated edges. Additionally, there are tiny, hair-like structures along the leaf edges and its surface, which are characteristic of certain plant species. The fine webbing around the eggs and holes may indicate the presence of spider mites or other pests that produce webbing. The time of day or the specific type of plant can't be determined from this

In [184]:
#I used the content of delphinium.txt and enbaled the web search to perform this + the image caption
result = agent.invoke('I started noticing some insects on my plant what should I do?'+caption)

In [185]:
#will get better when the the prompt imporve
result['output']

'To control insect pests on plants naturally, I can use methods such as maintaining healthy soil, using insect traps, adding beneficial insects, making homemade plant insecticide sprays, using vinegar solutions, and neem oil.'

This code still needs some more work
----------------------------------

In [110]:
import base64
import requests
from langchain.tools import BaseTool

class ImageCaptioner(BaseTool):
    name = "Image captioner"
    description = "Use this tool when given the path to an image that you would like to be described. "\
                  "This tool uses the GPT-4-vision-preview model of OpenAI's API and requires a valid API key"

    def __init__(self, api_key):
        self.api_key = api_key

    @staticmethod
    def _encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    def _run(self, image_path):
        # Getting the base64 string
        base64_image = self._encode_image(image_path)

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }

        payload = {
            "model": "gpt-4-vision-preview",
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": "Please provide a detailed description of the image, including the possible organisms it may feature, the time of day, and any relevant details. Remember, if the user provides information unrelated to agriculture, plants, gardens, etc., simply respond with 'nothing'."
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            "max_tokens": 300
        }

        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

        return response.json()['choices'][0]['message']['content']

    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")